In [30]:
import json
import urllib
import requests
import csv
import re
import calcTimeStamp

# Calculate the time and date for calucaltions so far

xy = calcTimeStamp.calcTimeStamp()
start, end = (xy[0], xy[1])
start = str(start)
end = str(end)

In [31]:
import datetime
from datetime import datetime
import dataFile

#
# Get data from the Tempest database for the new station
#

token = '877f6425-04a5-4f33-86e7-7123b7ef53d9'
protocol = 'https://'
urlSiteDevice = 'swd.weatherflow.com/swd/rest/observations/device/'
urlSiteStation = 'swd.weatherflow.com/swd/rest/observations/station/'
deviceID = '246921'
stationID = '95775'
preToken = '&token='
preStart = '?time_start='
preEnd = '&time_end='
start_time = start
end_time = end
dayOffset = '&day_offset=1'
format1 = '&format=csv'

#
# Put it together
# 

goGetDeviceSummary = (f'{protocol}{urlSiteDevice}{deviceID}{preStart}{start_time}{preEnd}{end_time}{format1}{preToken}{token}')
print(goGetDeviceSummary)
r =  requests.get(goGetDeviceSummary)
full_file = dataFile.dataFile_mac()

with open(full_file,'w') as fd:
     fd.write(r.text)

https://swd.weatherflow.com/swd/rest/observations/device/246921?time_start=1679803200&time_end=1679877215&format=csv&token=877f6425-04a5-4f33-86e7-7123b7ef53d9


In [32]:
import pandas as pd
import dataFile

#
# Read in the CSV file for processing in pandas
#

full_file = dataFile.dataFile_mac()
df = pd.read_csv(full_file, index_col=False)

pd.set_option('display.max_rows', 1440)
pd.set_option('display.max_columns', 35)
pd.set_option('display.width', 1500)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)

max_temp  = (df.sort_values(by='temperature', ascending=False))
max_T = max_temp.iloc[:1]
maxT = max_T['temperature'].values[0]
maxT = round((maxT*1.8) + 32)

min_temp  = (df.sort_values(by='temperature', ascending=True))
min_T = min_temp.iloc[:1]
minT = min_T['temperature'].values[0]
minT = round((minT*1.8) + 32)

tot_rain = df['precip'].sum()
totR = round((tot_rain*0.03937), 2)
nc_Rain = df['local_daily_precip_final'].iloc[-1]
ncR = round((nc_Rain*0.03937), 2)

recentT = (df['timestamp'].iloc[-1])
dt_object = datetime.fromtimestamp(recentT)
dt_object = str(dt_object)
datetime_obj = datetime.strptime(dt_object, '%Y-%m-%d %H:%M:%S')
lastTime = datetime_obj.strftime('%I:%M %p')

# write the data to a csv file with an html suffix

with open('/Users/jameshayes/HiLoRain_tempest.csv', 'w') as outfile:
    print(f'{maxT},{minT},{totR},{ncR},{lastTime}',file = outfile)